##Test

In [ ]:
!pip install torchmetrics
!pip install segmentation_models_pytorch

In [ ]:
import pandas as pd
import albumentations as albu
import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics import MeanSquaredError
from torchmetrics import Accuracy
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader, Dataset
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

In [ ]:
class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]

        if np.random.rand() < 0.5:
            label = 0.0
            pic_row = sound_df.loc[sound_df.type == 'clean'].iloc[0]
        else:
            label = 1.0
            pic_row = sound_df.loc[sound_df.type == 'noisy'].iloc[0]


        pic_path = os.path.join(self.source_folder, pic_row.get('folder'), pic_row.get('type'), str(pic_row.get('id')), sound)

        pic = np.expand_dims(np.load(pic_path).astype(float), 2)

        augmented = self.transforms(image=pic)

        return augmented['image'][0].unsqueeze(0).float(), torch.Tensor([label]).float()
        

In [ ]:
test_df = pd.read_csv('/content/meta_val.csv') # Specify test meta
test_transform = albu.Compose([albu.PadIfNeeded(480, 80),
            albu.RandomCrop(480, 80),
            albu.Resize(576, 96),
            ToTensorV2()])

test_dataset = SoundDataset(test_df, '/content/data', transforms = test_transform) # Specify data folder
test_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=1,
                                             num_workers=1)

device = ('cuda' if torch.cuda.is_available() else 'cpu')

params=dict(
            pooling='avg',
            dropout=0.2,
            activation='sigmoid',
            classes=1)
model = smp.Unet(encoder_name='resnet18', in_channels=1, aux_params = params).to(device)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/unet_21.pth')) # Specify weights
model.eval()

metric = Accuracy(0.5).to(device)

accuracy_metric = list()

with torch.no_grad():
  for x, y in tqdm(test_loader):

      x, y = x.to(device), y.to(device)
      pr_image, y_pred = model(x)
      accuracy_metric.append(metric(y_pred, y.type(torch.int64)).to('cpu').detach().numpy())

print('Accuracy = ', np.mean(accuracy_metric))

  0%|          | 0/2000 [00:00<?, ?it/s]

Accuracy =  0.991
